# Using interval arithmetic to calculate guaranteed bounds for $\pi$

In this notebook, we will see how using **interval arithmetic** simplifies calculating guaranteed bounds for $\pi$.

## Summing a series

We will repeat the calculation from the previous notebook, now using interval arithmetic, provided by the [`ValidatedNumerics.jl`](https://github.com/dpsanders/ValidatedNumerics.jl) package.

In [ ]:
using ValidatedNumerics

In [ ]:
setdisplay(:standard)  # abbreviated display of intervals

In [ ]:
S = Interval(0)

N = 10000
for i in 1:N
    S += 1 / i^2
end

S += 1/(N+1) .. 1/N  # interval bound on the remainder of the series

π_interval = √(6S)

Here we used an abbreviated display for the interval. Let's see the whole thing:

In [ ]:
setdisplay(:full)
π_interval

It's diameter (width) is

In [ ]:
diam(π_interval)

Thus, the result is correct to approximately 8 decimals.

In this calculation, we used the fact that arithmetic operations of intervals with numbers automatically promote the numbers to an interval:

In [ ]:
setdisplay(:full)  # full interval display
Interval(0) + 1/3^2

This is an interval containing the true real number $1/9$ (written `1//9` in Julia):

In [ ]:
big(1//9) ∈ convert(Interval{Float64}, 1/3^2)

Finally, we can check that the true value of $\pi$ is indeed inside our interval:

In [ ]:
big(pi) ∈ π_interval

## Calculating an area

Although the calculation above is simple, the derivation of the series itself is not. In this section, we will use a more natural way to calculate $\pi$, namely that the area of a circle of radius $r$ is $A(r) = \pi r^2$. We will calculate the area of one quadrant of a circle of radius $r=2$, which is equal to $\pi$:

In [ ]:
using Plots; gr()

In [ ]:
f(x) = √(4 - x^2)

In [ ]:
plot(f, 0, 2, aspect_ratio=:equal, fill=(0, :orange), alpha=0.2, label="")

The circle of radius $r=2$ is given by $x^2 + y^2 = 2^2 = 4$, so 

$$\pi = \frac{1}{4} A(2) = \int_{x=0}^2 y(x) \, dx = \int_{x=0}^2 \sqrt{4 - x^2}.$$

In calculus, we learn that we can approximate integrals using **Riemann sums**. Interval arithmetic allows us to make these Riemann sums **rigorous** in a very simple way, as follows.

We split up the $x$ axis into intervals, for example of equal width:

In [ ]:
function make_intervals(N=10)
    xs = linspace(0, 2, N+1)
    return [xs[i]..xs[i+1] for i in 1:length(xs)-1]
end

intervals = make_intervals()

Given one of those intervals, we evaluate the function of interest:

In [ ]:
II = intervals[1]

In [ ]:
f(II)

The result is an interval that is **guaranteed to contain** the true range of the function $f$ over that interval. So the lower and upper bounds of the intervals may be used as lower and upper bounds of the height of the box in a Riemann integral:

In [ ]:
intervals = make_intervals(30)

p = plot(aspect_ratio=:equal)
for X in intervals
    Y = f(X)
    
    plot!(IntervalBox(X, Interval(0, Y.lo)), c=:blue, label="", alpha=0.1)
    plot!(IntervalBox(X, Interval(Y.lo, Y.hi)), c=:red, label="", alpha=0.1)
end

plot!(f, 0, 2)

p

Now we just sum up the areas:

In [ ]:
N = 20
intervals = make_intervals(N)

width = 2/N
width * sum(√(4 - X^2) for X in intervals)

As we increase the number of sub-intervals, the approximation gets better and better:

In [ ]:
setdisplay(:standard, sigfigs=5)

println("N \t area interval \t \t diameter")
for N in 50:50:1000
    intervals = make_intervals(N)
    area = (2/N) * sum(√(4 - X^2) for X in intervals)
            
    println("$N \t $area \t $(diam(area))")
end